In [1]:
from src.ai import createOpenAIClient

client = createOpenAIClient()

In [14]:
prompts = [
    {
        'gpt_prompt': 'Lag en kort koselig julehilsen til min gode kollega, Mikal 2',
        'gpt_system_message': 'Jeg heter Ole Martin og er en software developer som ønsker Mikal en god jul',
        'dalle_prompt': 'Snowman',
        'epost': 'mikal.stapnes@twoday.com',
        'avsender': 'Mikal'
    },
    {
        'gpt_prompt': 'Lag en kort koselig rap til min gode kollega, Mikal',
        'gpt_system_message': 'Jeg heter Ole Martin og er en software developer som ønsker Mikal en god jul',
        'dalle_prompt': 'Snowman',
        'epost': 'mikal.stapnes@twoday.com',
        'avsender': 'Mikal 2'
    }
]

In [15]:
from src.ai import generate_text

for p in prompts:
    text_content = generate_text(client, p['gpt_prompt'], p['gpt_system_message'])

    p['text_content'] = text_content

In [16]:
from src.ai import generate_image_b64
from PIL import Image

images = [generate_image_b64(client, p['dalle_prompt']) for p in prompts]

In [17]:
import requests 
from src.image_text import put_text_on_image
from src.storeblob import upload_file_to_blob
import os
from dotenv import load_dotenv

load_dotenv()
connection_string = os.getenv("ABS_CS")
key = os.getenv("ABS_KEY")

urls = []

for i, image in enumerate(images):

    file = requests.get(image.data[0].url)
    image_path = f'./images/{i}.png'

    with open(image_path, "wb") as image_file:
        image_file.write(file.content)

    output_path = f'./images/{i}_texted.png'
    image = put_text_on_image(image_path, text_content, output_path, font_path="./resources/font.ttf")

    url = upload_file_to_blob(output_path, connection_string, key, 'images', output_path.split('/')[-1])

    urls.append(url)

In [18]:
print(urls)

['https://stfagkveldsendgrid.blob.core.windows.net/images/0_texted.png?se=2023-12-11T15%3A43%3A24Z&sp=r&sv=2023-11-03&ss=b&srt=o&sig=JKz9pOmuH97NF/UZnWZN/UDACtyXty7qhb2vlxixj18%3D', 'https://stfagkveldsendgrid.blob.core.windows.net/images/1_texted.png?se=2023-12-11T15%3A43%3A25Z&sp=r&sv=2023-11-03&ss=b&srt=o&sig=L/JJ/OLYzhCaOHwyhJ6ZigeDoKnnQmRg64Pl2S8BQoU%3D']


In [19]:
from src.graph import get_secret_access_token
access_token = get_secret_access_token()

{'token_type': 'Bearer', 'expires_in': 3599, 'ext_expires_in': 3599, 'access_token': 'eyJ0eXAiOiJKV1QiLCJub25jZSI6ImxJVE92anpJRkhKT25Sc2Nuc3ZzblRub0k5UUZtSFpBeW5aMEo0ZWt5OWsiLCJhbGciOiJSUzI1NiIsIng1dCI6IlQxU3QtZExUdnlXUmd4Ql82NzZ1OGtyWFMtSSIsImtpZCI6IlQxU3QtZExUdnlXUmd4Ql82NzZ1OGtyWFMtSSJ9.eyJhdWQiOiJodHRwczovL2dyYXBoLm1pY3Jvc29mdC5jb20iLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC9iYTlkNzMwNS0zMGVkLTQyY2EtOGI1NC02NWQ5OGQwZjJhMjkvIiwiaWF0IjoxNzAyMjE5MTA1LCJuYmYiOjE3MDIyMTkxMDUsImV4cCI6MTcwMjIyMzAwNSwiYWlvIjoiRTJWZ1lJaC9ybFo0bHYxcGR1ejV6ZW1iNUZQa0FBPT0iLCJhcHBfZGlzcGxheW5hbWUiOiJBenVyZU1haWxlcjMiLCJhcHBpZCI6ImYxODQ3OWM0LTBkNGUtNDYyMC1hNDM1LWFiM2ZlMDZlZTI3NSIsImFwcGlkYWNyIjoiMSIsImlkcCI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0L2JhOWQ3MzA1LTMwZWQtNDJjYS04YjU0LTY1ZDk4ZDBmMmEyOS8iLCJpZHR5cCI6ImFwcCIsIm9pZCI6IjQ5Y2ViYTg5LTMwOGUtNGRjOC1iNDJhLThlZDc2YTUyMDBmZiIsInJoIjoiMC5BWUVBQlhPZHV1MHd5a0tMVkdYWmpROHFLUU1BQUFBQUFBQUF3QUFBQUFBQUFBQ0JBQUEuIiwicm9sZXMiOlsiTWFpbC5TZW5kIl0sInN1YiI6IjQ5Y2ViYTg5LTMwOGUtNGRjO

In [20]:
userId = "mikalst@mikalst.onmicrosoft.com"
endpoint = f'https://graph.microsoft.com/v1.0/users/{userId}/sendmail'

In [21]:
import random

order = list(range(len(prompts)))

random.shuffle(order)

In [22]:
import requests
import src.christmasmail as christmasmail

def sendmail(i, j):
    mottaker = prompts[j]['avsender']
    avsender = prompts[i]['avsender']
    overskrift = f'Kjære {mottaker}'
    url = urls[i]
    toUserEmail = prompts[j]['epost']

    email_msg = christmasmail.generate_christmas_email(overskrift, url, avsender, toUserEmail)

    r = requests.post(endpoint, headers={'Authorization': f'Bearer {access_token}'}, json=email_msg)

    if not(r.ok):
        print(r.json())

for i in range(len(order)-1):
    sendmail(i, i+1)
sendmail(len(order)-1, 0)